In [129]:
import random
import torch
import numpy as np

In [141]:
from microservice.models.model_b import SimpleNN
from models.utils import load_default_data, calculate_expenses
from microservice.features import extract_users_data, prepare_data_for_predict
from sklearn.metrics import roc_auc_score as roc
from microservice.models.model_b import NeuralNetworkRegressor
from models.train_utils_nn import train


In [142]:
seed = 213769420
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [143]:
train_sessions, test, products, users, deliveries = load_default_data()
targets = calculate_expenses(test, products, users)
targets

,user_id,expenses
0,102,0.00
1,103,554.77
2,104,2332.01
3,105,0.00
4,106,0.00
...,...,...
195,297,109.00
196,298,2399.00
197,299,0.00
198,300,0.00


In [144]:
# targets['expenses'] = targets['expenses'].apply(lambda b: 1 if b > 5000 else 0) binarization
targets['expenses'] = targets['expenses'].apply(lambda x: x/100)

In [145]:
targets

,user_id,expenses
0,102,0.0000
1,103,5.5477
2,104,23.3201
3,105,0.0000
4,106,0.0000
...,...,...
195,297,1.0900
196,298,23.9900
197,299,0.0000
198,300,0.0000


In [146]:
some_net = NeuralNetworkRegressor()
users_data = extract_users_data(train_sessions, users, products)
users_data

,user_id,expenses,products_bought,events_number,sessions_number,average_discount,average_discount_on_bought,city
0,139,23400.85,49,255,66,11.019608,10.816327,Kraków
1,242,44518.18,58,327,86,11.223242,10.603448,Radom
2,108,16312.03,23,129,32,9.418605,9.347826,Radom
3,143,7214.08,15,84,26,11.250000,11.666667,Kraków
4,140,32333.55,43,306,83,10.130719,9.651163,Poznań
...,...,...,...,...,...,...,...,...
195,289,0.00,0,7,1,20.000000,0.000000,Warszawa
196,225,109.00,1,2,1,10.000000,10.000000,Warszawa
197,152,0.00,0,5,2,2.000000,0.000000,Poznań
198,162,78.96,2,6,2,13.333333,12.500000,Szczecin


In [136]:
train(some_net, users_data, targets)

Epoch 0 loss 1.73e+02
Epoch 1 loss 2.84e+02
Epoch 2 loss 1.87e+02
Epoch 3 loss 2.16e+02
Epoch 4 loss 1.85e+02
Epoch 5 loss 2.16e+02
Epoch 6 loss 1.92e+02
Epoch 7 loss 1.9e+02
Epoch 8 loss 1.81e+02
Epoch 9 loss 2.47e+02
Epoch 10 loss 2.23e+02
Epoch 11 loss 2.66e+02
Epoch 12 loss 2.25e+02
Epoch 13 loss 2.03e+02
Epoch 14 loss 1.9e+02
Epoch 15 loss 1.94e+02
Epoch 16 loss 1.77e+02
Epoch 17 loss 2.07e+02
Epoch 18 loss 2.86e+02
Epoch 19 loss 2.17e+02
Epoch 20 loss 1.89e+02
Epoch 21 loss 2.28e+02
Epoch 22 loss 2.16e+02
Epoch 23 loss 2.54e+02
Epoch 24 loss 2.06e+02
Epoch 25 loss 2.7e+02
Epoch 26 loss 1.76e+02
Epoch 27 loss 2.57e+02
Epoch 28 loss 2.73e+02
Epoch 29 loss 2.68e+02
Epoch 30 loss 1.81e+02
Epoch 31 loss 1.89e+02
Epoch 32 loss 2.15e+02
Epoch 33 loss 2.17e+02
Epoch 34 loss 1.98e+02
Epoch 35 loss 1.78e+02
Epoch 36 loss 2.97e+02
Epoch 37 loss 1.75e+02
Epoch 38 loss 1.83e+02
Epoch 39 loss 2.67e+02
Epoch 40 loss 1.8e+02
Epoch 41 loss 1.94e+02
Epoch 42 loss 1.81e+02
Epoch 43 loss 2.57e+02
Ep

In [137]:
extracted_users_data = extract_users_data(train_sessions, users, products)
x, cat_x = prepare_data_for_predict(extracted_users_data)
x = torch.from_numpy(x.values).float()
cat_x = torch.from_numpy(cat_x.values).float()
some_net.eval()
out = some_net(x, cat_x).squeeze()
out = out.detach().numpy()
out

array([2.5522861 , 6.670182  , 1.1285622 , 0.99783105, 4.2777514 ,
       2.145917  , 4.229818  , 2.3139265 , 0.45462483, 1.608789  ,
       2.7811337 , 0.5814664 , 0.43894744, 2.1252663 , 3.6308777 ,
       1.0397326 , 1.3970548 , 2.6281579 , 0.41820455, 2.0558407 ,
       1.8769951 , 2.0723605 , 1.0095036 , 1.3822054 , 0.9210697 ,
       1.186477  , 3.5009465 , 1.5893373 , 2.0874112 , 0.609076  ,
       0.8767379 , 0.46969908, 0.39046878, 1.7755748 , 1.0801693 ,
       1.6740756 , 0.6364358 , 1.2792948 , 6.878815  , 0.5609837 ,
       0.6224723 , 3.3456028 , 3.7511055 , 2.8830326 , 2.0223958 ,
       2.2166443 , 1.790338  , 3.510706  , 1.5492185 , 0.37559396,
       3.4398756 , 1.8429571 , 3.5136185 , 1.8530477 , 2.0842972 ,
       2.0739944 , 0.52779543, 1.1062891 , 0.88381904, 1.871858  ,
       3.154615  , 1.0335704 , 0.48410177, 0.574937  , 1.2008716 ,
       1.2551936 , 1.309692  , 2.1514757 , 1.7297982 , 1.9524814 ,
       2.0271084 , 1.4710511 , 0.99491006, 0.5781651 , 2.28928

In [139]:
# out_array = {int(user_id): out[i] > 0.3 for i, user_id in enumerate(extracted_users_data["user_id"].to_list())} binarization
out_array = {int(user_id): out[i] for i, user_id in enumerate(extracted_users_data["user_id"].to_list())}
out_array

{139: 2.5522861,
 242: 6.670182,
 108: 1.1285622,
 143: 0.99783105,
 140: 4.2777514,
 202: 2.145917,
 270: 4.229818,
 212: 2.3139265,
 189: 0.45462483,
 205: 1.608789,
 255: 2.7811337,
 215: 0.5814664,
 273: 0.43894744,
 281: 2.1252663,
 227: 3.6308777,
 222: 1.0397326,
 235: 1.3970548,
 171: 2.6281579,
 188: 0.41820455,
 135: 2.0558407,
 252: 1.8769951,
 238: 2.0723605,
 145: 1.0095036,
 286: 1.3822054,
 157: 0.9210697,
 265: 1.186477,
 127: 3.5009465,
 159: 1.5893373,
 119: 2.0874112,
 274: 0.609076,
 160: 0.8767379,
 105: 0.46969908,
 261: 0.39046878,
 185: 1.7755748,
 183: 1.0801693,
 192: 1.6740756,
 186: 0.6364358,
 102: 1.2792948,
 124: 6.878815,
 282: 0.5609837,
 210: 0.6224723,
 125: 3.3456028,
 272: 3.7511055,
 104: 2.8830326,
 115: 2.0223958,
 136: 2.2166443,
 285: 1.790338,
 133: 3.510706,
 187: 1.5492185,
 175: 0.37559396,
 161: 3.4398756,
 167: 1.8429571,
 213: 3.5136185,
 292: 1.8530477,
 256: 2.0842972,
 278: 2.0739944,
 263: 0.52779543,
 148: 1.1062891,
 200: 0.8838190

In [140]:
targets.values

array([[1.02000e+02, 0.00000e+00],
       [1.03000e+02, 5.54770e+00],
       [1.04000e+02, 2.33201e+01],
       [1.05000e+02, 0.00000e+00],
       [1.06000e+02, 0.00000e+00],
       [1.07000e+02, 7.89000e-01],
       [1.08000e+02, 0.00000e+00],
       [1.09000e+02, 1.19980e+00],
       [1.10000e+02, 1.07900e+01],
       [1.11000e+02, 6.11800e-01],
       [1.12000e+02, 3.07320e+00],
       [1.13000e+02, 0.00000e+00],
       [1.14000e+02, 0.00000e+00],
       [1.15000e+02, 0.00000e+00],
       [1.16000e+02, 9.39440e+00],
       [1.17000e+02, 0.00000e+00],
       [1.18000e+02, 0.00000e+00],
       [1.19000e+02, 0.00000e+00],
       [1.20000e+02, 0.00000e+00],
       [1.21000e+02, 1.83196e+01],
       [1.22000e+02, 2.42602e+01],
       [1.23000e+02, 2.19990e+01],
       [1.24000e+02, 2.87002e+01],
       [1.25000e+02, 1.79000e-01],
       [1.26000e+02, 0.00000e+00],
       [1.27000e+02, 2.41701e+01],
       [1.28000e+02, 9.99900e-01],
       [1.29000e+02, 2.19990e+01],
       [1.30000e+02,

In [56]:
numerator = 0
for row in targets.values[:]:
    if row[1] == out_array[row[0]]:
        print(row[0])
        numerator += 1

102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
165
166
167
168
169
170
171
172
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
267
268
269
270
271
272
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301


In [57]:
some_net(x, cat_x).detach()

tensor([[0.0927],
        [0.0727],
        [0.1011],
        [0.0847],
        [0.0827],
        [0.0970],
        [0.0828],
        [0.0978],
        [0.0843],
        [0.0618],
        [0.0891],
        [0.0878],
        [0.0957],
        [0.0425],
        [0.0880],
        [0.1005],
        [0.0806],
        [0.0935],
        [0.0873],
        [0.0487],
        [0.0544],
        [0.0928],
        [0.0856],
        [0.0696],
        [0.0872],
        [0.0833],
        [0.0868],
        [0.0696],
        [0.0496],
        [0.0898],
        [0.1002],
        [0.0917],
        [0.0890],
        [0.1001],
        [0.0772],
        [0.0677],
        [0.0891],
        [0.0797],
        [0.0741],
        [0.0906],
        [0.0902],
        [0.0884],
        [0.0882],
        [0.0929],
        [0.0480],
        [0.0970],
        [0.0597],
        [0.0915],
        [0.0713],
        [0.0898],
        [0.0876],
        [0.0526],
        [0.0895],
        [0.0939],
        [0.0432],
        [0

In [58]:
roc(targets.expenses, some_net(x, cat_x).detach())

0.6915807560137457

In [59]:
torch.save(some_net.state_dict(), "../models/parameters/dummy_test")